In [ ]:
!git clone https://github.com/KushajveerSingh/ds_cup

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [ ]:
smallestF =  np.finfo('float').eps # smallest float value (numpy), will be used later

In [ ]:
path = Path('../data_orig')
# path = Path('ds_cup/orig_data')
orig_train_df = pd.read_csv(path/'train.csv')
orig_valid_df = pd.read_csv(path/'valid.csv')
orig_test_df = pd.read_csv(path/'test.csv')

In [ ]:
orig_train_df

In [ ]:
def print_size(df, name):
    counts = df['Default_ind'].value_counts()
    print(f'{name} dataset')
    print(f'Num 0 values = {counts[0]}')
    print(f'Num 1 values = {counts[1]}')
    print()

In [ ]:
print_size(orig_train_df, 'train')
print_size(orig_valid_df, 'valid')
print_size(orig_test_df, 'test')

In [ ]:
orig_train_df.isnull().sum()

In [ ]:
#def binColumn(col):
def binColumn(col,threshArr):
    assert col.isnull().all() != True, "All values are null?"
    def binVal(val):
        loc = 1
        for x in threshArr:
            if(val<=x):
                return loc
            else:
                loc = loc+1
        return loc
    theCol = col.copy()
    threshArr = np.sort(np.array(threshArr))
    #Replace Nulls with 0 if any
    theBinnedCol = []
    for val in theCol:
        if(not np.isnan(val)):
            theBinnedCol.extend([binVal(val)])
        else:
            theBinnedCol.extend([0])
        #theCol.update(pd.Series([], index=[ind]))
    return pd.Series(theBinnedCol,index=col.keys())

In [ ]:
def createThreshDict(colName,Arr):
    _dict = {}
    _dict['colName'] = colName
    _dict['ThreshArr'] = Arr
    return _dict
DFThresh =  pd.DataFrame(columns=createThreshDict("",[]).keys())

- Going through all columns one by one and applying binning and solving Null Problems

In [ ]:
#orig_train_df = orig_train_df.sample(frac=1) #Shuffling the dataset before using
#orig_valid_df = orig_valid_df.sample(frac=1) #Shuffling the dataset before using
#orig_test_df = orig_test_df.sample(frac=1) #Shuffling the dataset before using
train_df = pd.DataFrame(columns=orig_train_df.columns)
#valid_df = pd.DataFrame(columns=orig_valid_df.columns)
#test_df = pd.DataFrame(columns=orig_valid_df.columns)

In [ ]:
orig_train_df.columns

## - tot_credit_debt

In [ ]:
_col = 'tot_credit_debt'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - avg_card_debt

In [ ]:
cutoff_avg_card_debt = (19960.610000 + 99999.0)/2
cutoff_avg_card_debt

In [ ]:
#13167.825 (cutoff) is max value of lower cluster and 99999 is value of next one
orig_train_df[orig_train_df.avg_card_debt > cutoff_avg_card_debt]['avg_card_debt'].describe(),orig_train_df[orig_train_df.avg_card_debt < cutoff_avg_card_debt]['avg_card_debt'].describe()

In [ ]:
print('Column = avg_card_debt')
fig, ax = plt.subplots(1,2)
ax[0].plot(orig_train_df[orig_train_df.avg_card_debt > cutoff_avg_card_debt]['avg_card_debt'],'ro',marker='*',)
ax[1].plot(orig_train_df[orig_train_df.avg_card_debt < cutoff_avg_card_debt]['avg_card_debt'],'ro',marker='*',)
plt.show()

In [ ]:
_col = 'avg_card_debt'
_thresh = orig_train_df[orig_train_df.avg_card_debt < cutoff_avg_card_debt]['avg_card_debt'].quantile([0.2,0.4,0.6,0.8]).to_numpy()
_thresh = np.append(_thresh, [cutoff_avg_card_debt])
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - credit_age

In [ ]:
_col = 'credit_age'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - credit_good_age

In [ ]:
_col = 'credit_good_age'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - card_age

In [ ]:
_col = 'card_age'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - non_mtg_acc_past_due_12_months_num

In [ ]:
print(orig_train_df.non_mtg_acc_past_due_12_months_num.value_counts()) #Already binned
train_df['non_mtg_acc_past_due_12_months_num'] = orig_train_df['non_mtg_acc_past_due_12_months_num'].copy()

## - non_mtg_acc_past_due_6_months_num

In [ ]:
print(orig_train_df.non_mtg_acc_past_due_6_months_num.value_counts()) #Already Binned
train_df['non_mtg_acc_past_due_6_months_num'] = orig_train_df['non_mtg_acc_past_due_6_months_num'].copy()

## - mortgages_past_due_6_months_num

In [ ]:
print(orig_train_df.mortgages_past_due_6_months_num.value_counts()) #Already Binned
train_df['mortgages_past_due_6_months_num'] = orig_train_df['mortgages_past_due_6_months_num'].copy()

## - credit_past_due_amount

In [ ]:
orig_train_df.credit_past_due_amount.describe() #Binning Problem, 0 must be seperate

In [ ]:
orig_train_df[orig_train_df.credit_past_due_amount > 0].credit_past_due_amount.describe() #Binning Problem, 0 must be seperate

In [ ]:
plt.plot(orig_train_df.credit_past_due_amount[orig_train_df.credit_past_due_amount > 0],'ro',marker='*')
plt.show()

In [ ]:
_col = 'credit_past_due_amount'
_thresh = orig_train_df[orig_train_df.credit_past_due_amount > 0][_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
_thresh = np.insert(_thresh, 0, 0+smallestF)
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - inq_12_month_num

In [ ]:
_col = 'inq_12_month_num'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - card_inq_24_month_num

In [ ]:
_col = 'card_inq_24_month_num'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - card_open_36_month_num  

In [ ]:
_col = 'card_open_36_month_num'
print(orig_train_df[_col].value_counts()) #Already Binned
train_df[_col] = orig_train_df[_col].copy()

## - auto_open_36_month_num

In [ ]:
_col = 'auto_open_ 36_month_num'
print(orig_train_df[_col].value_counts()) #Already Binned
train_df[_col] = orig_train_df[_col].copy()

## - uti_card

In [ ]:
_col = 'uti_card'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - uti_50plus_pct

In [ ]:
_col = 'uti_50plus_pct'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - uti_max_credit_line

In [ ]:
_col = 'uti_max_credit_line'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - uti_card_50plus_pct

In [ ]:
_col = 'uti_card_50plus_pct'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - ind_acc_XYZ

In [ ]:
_col = 'ind_acc_XYZ'
print(orig_train_df[_col].value_counts()) #Already Binned
train_df[_col] = orig_train_df[_col].copy()

## - rep_income

In [ ]:
_col = 'rep_income'
_thresh = orig_train_df[_col].quantile([0.2,0.4,0.6,0.8]).to_numpy()
DFThresh = DFThresh.append(createThreshDict(_col,
    _thresh),
    ignore_index=True) 
train_df[_col] = binColumn(orig_train_df[_col],_thresh)

## - States

In [ ]:
_col = 'States'
#
train_df.drop(columns=[_col],inplace=True,errors='ignore')
train_df.insert(0,_col,orig_train_df[_col].copy())
#
enc = OneHotEncoder(handle_unknown='ignore',sparse = False)
SM = enc.fit_transform(orig_train_df[_col].to_numpy().reshape(-1,1)) #Got it!
for ind,cat in zip(range(0,len(enc.categories_[0])),enc.categories_[0]):
    train_df.drop(columns=['is'+str(cat)],inplace=True,errors='ignore')
    train_df.insert(0,'is'+str(cat), SM[:,ind]) # Inserted column names are isAK, isAL, isDC..etc. 
   #And it will contain 0 if that is not the state and 1 if that is the state,
#train_df.drop(columns=[_col],inplace=True) #Remove States Column

## - Default_ind (Output Column)

In [ ]:
_col = 'Default_ind'
print(orig_train_df[_col].value_counts()) #Already Binned
train_df[_col] = orig_train_df[_col].copy()

In [ ]:
train_df

In [ ]:
DFThresh

In [ ]:
train_df.to_csv(path/'train_binned.csv',index=None)
DFThresh.to_csv(path/'Thresholds.csv',index=None)

In [ ]:
train_df

In [ ]:
orig_train_df

In [ ]:
orig_train_df.groupby(by=['States']).std()['rep_income']